In [1]:
# Install these packages if running from colab
!pip install tensorflow-datasets --quiet
!pip install pydot --quiet
!pip install transformers --quiet

# install huggingface datasets
!pip install datasets --quiet

! pip install rouge-score nltk --quiet
! pip install huggingface_hub --quiet

     |████████████████████████████████| 5.5 MB 27.9 MB/s 
     |████████████████████████████████| 7.6 MB 59.3 MB/s 
     |████████████████████████████████| 163 kB 95.3 MB/s 
     |████████████████████████████████| 441 kB 27.1 MB/s 
     |████████████████████████████████| 95 kB 6.2 MB/s 
     |████████████████████████████████| 212 kB 90.8 MB/s 
     |████████████████████████████████| 115 kB 48.5 MB/s 
     |████████████████████████████████| 127 kB 97.7 MB/s 
     |████████████████████████████████| 115 kB 87.1 MB/s 


In [2]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds

import sklearn as sk
import os
import nltk
from nltk.data import find

import matplotlib.pyplot as plt

import re

#let's make longer output readable without scrolling
from pprint import pprint

# the toxic parallel dataset, with rouge metric
from datasets import load_dataset, load_from_disk, load_metric, DatasetDict

In [3]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# define paths
dataset_path = 'drive/MyDrive/Colab Notebooks/w266_project_data'
csv_path = 'drive/MyDrive/Colab Notebooks/w266_project_predictions/'

#### Change these variables as needed for different model and different file name

In [5]:
# change these variables for different models
output_file_name = 'bart_xsum_zsl.csv'
model_checkpoint = "facebook/bart-large-xsum"

### Import and examine our dataset

In [6]:
# load the dataset
dataset = load_from_disk(dataset_path)

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['en_toxic_comment', 'en_neutral_comment'],
        num_rows: 17789
    })
    test: Dataset({
        features: ['en_toxic_comment', 'en_neutral_comment'],
        num_rows: 989
    })
    valid: Dataset({
        features: ['en_toxic_comment', 'en_neutral_comment'],
        num_rows: 988
    })
})

In [8]:
metric = load_metric("rouge")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  """Entry point for launching an IPython kernel.


In [9]:
pd.DataFrame(dataset['train'])

,en_toxic_comment,en_neutral_comment
0,u 'd be surprised all the shit u think about w...,You would be supriesd all the things you think...
1,plenty of holocaust deniers actually believe t...,Plenty of Holocaust deniers actually believe t...
2,holy fuck i 'm so going,i'm so going
3,i 've driven by lots of cattle farms but holy ...,i 've driven by lots of cattle farms but this
4,so much better this shit to soft for me,So much better this is to soft for me.
...,...,...
17784,"but , thats exactly what a couple of you idiot...","But, that’s exactly what a couple of you have ..."
17785,he probably assessed the situation and decided...,He probably assessed the situation and decided...
17786,brah im fucked up over here .,Brah I am feeling bad over here
17787,shit with my friends and not being able to tru...,with my friends and not being able to trust th...


## load BART xsum large pretrain model

In [11]:
from transformers import BartTokenizer, TFBartForConditionalGeneration

model = TFBartForConditionalGeneration.from_pretrained(model_checkpoint)
tokenizer = BartTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBartForConditionalGeneration.

All the layers of TFBartForConditionalGeneration were initialized from the model checkpoint at facebook/bart-large-xsum.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

In [12]:
model.summary()

Model: "tf_bart_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (TFBartMainLayer)     multiple                  406290432 
                                                                 
 final_logits_bias (BiasLaye  multiple                 50264     
 r)                                                              
                                                                 
Total params: 406,340,696
Trainable params: 406,290,432
Non-trainable params: 50,264
_________________________________________________________________


### Preprocess: Encode detox dataset with BART xsum tokenizer

In [13]:
## Encode detox train_texts with BART tokenizer
max_length = 25
#max_label_length = 128

def preprocess_function(examples):
    inputs = [doc for doc in examples['en_toxic_comment']]
    model_inputs = tokenizer(inputs, max_length=max_length, truncation=True, padding=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples['en_neutral_comment'], max_length=max_length, truncation=True, padding=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [14]:
# test the preprocess function
preprocess_function(dataset['train'][:2])

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3547: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


{'input_ids': [[0, 257, 128, 417, 28, 3911, 70, 5, 15328, 1717, 206, 59, 77, 1717, 1236, 687, 579, 2582, 179, 89, 2], [0, 2911, 24997, 9, 18701, 43328, 3069, 4733, 888, 679, 5, 5373, 15328, 51, 224, 479, 2, 1, 1, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]], 'labels': [[0, 1185, 74, 28, 28836, 4458, 417, 70, 5, 383, 47, 206, 59, 77, 47, 32, 95, 2828, 89, 4, 2], [0, 16213, 24997, 9, 13903, 3069, 4733, 888, 679, 5, 383, 51, 224, 4, 2, 1, 1, 1, 1, 1, 1]]}

In [15]:
# tokenize the detox dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True)

### no fine-tuning, zero-shot. Try on one example

In [16]:
input_tokenized = tokenizer([dataset['test']['en_toxic_comment'][1]], return_tensors="tf").input_ids
summary_ids = model.generate(input_tokenized, num_beams=2, min_length=0, max_length=25)

In [17]:
prediction = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
reference = [dataset['test']['en_neutral_comment'][1]]

In [18]:
print([dataset['test']['en_toxic_comment'][1]])
print(prediction)
print(reference)

['all she has to do is smile and keep her cool with slobbering old wrinkled hubby .']
['Theresa May has been praised for the way she handled the row between her husband and her ex-partner,']
['All she has to do is smile and keep her cool with old aged hubby']


In [19]:
rouge_results = metric.compute(predictions=prediction,
                               references=reference)
pprint(rouge_results, compact=True)  

{'rouge1': AggregateScore(low=Score(precision=0.21052631578947367, recall=0.26666666666666666, fmeasure=0.23529411764705882), mid=Score(precision=0.21052631578947367, recall=0.26666666666666666, fmeasure=0.23529411764705882), high=Score(precision=0.21052631578947367, recall=0.26666666666666666, fmeasure=0.23529411764705882)),
 'rouge2': AggregateScore(low=Score(precision=0.0, recall=0.0, fmeasure=0.0), mid=Score(precision=0.0, recall=0.0, fmeasure=0.0), high=Score(precision=0.0, recall=0.0, fmeasure=0.0)),
 'rougeL': AggregateScore(low=Score(precision=0.15789473684210525, recall=0.2, fmeasure=0.17647058823529413), mid=Score(precision=0.15789473684210525, recall=0.2, fmeasure=0.17647058823529413), high=Score(precision=0.15789473684210525, recall=0.2, fmeasure=0.17647058823529413)),
 'rougeLsum': AggregateScore(low=Score(precision=0.15789473684210525, recall=0.2, fmeasure=0.17647058823529413), mid=Score(precision=0.15789473684210525, recall=0.2, fmeasure=0.17647058823529413), high=Score(

### zero-shot learning (ZSL) on the test set

In [20]:
test_predictions = []
test_references = []
test_len = len(dataset['test']['en_toxic_comment'])

for i in range(test_len):
  input_tokenized = tokenizer([dataset['test']['en_toxic_comment'][i]], return_tensors="tf").input_ids
  summary_ids = model.generate(input_tokenized, num_beams=2, min_length=0, max_length=25)
  
  prediction = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
  reference = [dataset['test']['en_neutral_comment'][i]]
  
  test_predictions.insert(i, (prediction))
  test_references.insert(i, (reference))

  if i % 50 == 0:
    print('complete', i, '/', test_len)
#print(len(val_references))

complete 0 / 989
complete 50 / 989
complete 100 / 989
complete 150 / 989
complete 200 / 989
complete 250 / 989
complete 300 / 989
complete 350 / 989
complete 400 / 989
complete 450 / 989
complete 500 / 989
complete 550 / 989
complete 600 / 989
complete 650 / 989
complete 700 / 989
complete 750 / 989
complete 800 / 989
complete 850 / 989
complete 900 / 989
complete 950 / 989


In [21]:
# dictionary of lists  
dict = {'test_predictions': test_predictions, 'test_references': test_references}  
       
df = pd.DataFrame(dict) 

In [22]:
# saving the output dataframe to a csv file
# output_file_name = 'bart_large_zsl.csv'
df.to_csv(csv_path + output_file_name, index = False) 

In [23]:
# read data from csv file
df_bart_predictions = pd.read_csv(csv_path + output_file_name)

# display data frame
print(df_bart_predictions.head())

# display shape
print(df_bart_predictions.shape)

# display columns
print(df_bart_predictions.columns)

                                    test_predictions  \
0  ['The BBC has released a video of the moment t...   
1  ['Theresa May has been praised for the way she...   
2  ['A few years ago I wrote a post saying that i...   
3  ["Thats what I thought about it, so I'm going ...   
4  ["The BBC has been given a special report on t...   

                                     test_references  
0                    ['The 6161 view must be funny']  
1  ['All she has to do is smile and keep her cool...  
2  ['can we apply that litmus test to muslims tha...  
3                        ["That's it! I'm joining."]  
4    ['The residents of Davis are not so pleasant.']  
(989, 2)
Index(['test_predictions', 'test_references'], dtype='object')


In [24]:
rouge_results = metric.compute(predictions=df_bart_predictions['test_predictions'],
                               references=df_bart_predictions['test_references'])
pprint(rouge_results, compact=True)  

{'rouge1': AggregateScore(low=Score(precision=0.1902307737370551, recall=0.27319891750350006, fmeasure=0.21207492526676372), mid=Score(precision=0.20586197805238438, recall=0.2893601356205872, fmeasure=0.2272918439444932), high=Score(precision=0.22132477104793608, recall=0.30670232216726717, fmeasure=0.24273838824260613)),
 'rouge2': AggregateScore(low=Score(precision=0.0893992494028211, recall=0.1094952536035144, fmeasure=0.09269626759019453), mid=Score(precision=0.10313025794017171, recall=0.12518242891019915, fmeasure=0.10701958643779046), high=Score(precision=0.11643086105191729, recall=0.14002242477629814, fmeasure=0.12021998944374189)),
 'rougeL': AggregateScore(low=Score(precision=0.179613281959109, recall=0.2561949713390042, fmeasure=0.19968906911843712), mid=Score(precision=0.19448035654995338, recall=0.2722335046298996, fmeasure=0.2139995591164451), high=Score(precision=0.2101469073765577, recall=0.2892010780055225, fmeasure=0.23008358296719814)),
 'rougeLsum': AggregateScore

In [25]:
pd.DataFrame.from_dict(rouge_results)

,rouge1,rouge2,rougeL,rougeLsum
0,"(0.1902307737370551, 0.27319891750350006, 0.21...","(0.0893992494028211, 0.1094952536035144, 0.092...","(0.179613281959109, 0.2561949713390042, 0.1996...","(0.17995231875630316, 0.25453779057210496, 0.1..."
1,"(0.20586197805238438, 0.2893601356205872, 0.22...","(0.10313025794017171, 0.12518242891019915, 0.1...","(0.19448035654995338, 0.2722335046298996, 0.21...","(0.19442395404911617, 0.2715578887647925, 0.21..."
2,"(0.22132477104793608, 0.30670232216726717, 0.2...","(0.11643086105191729, 0.14002242477629814, 0.1...","(0.2101469073765577, 0.2892010780055225, 0.230...","(0.20976736351717246, 0.28732820850839363, 0.2..."


In [26]:
pd.concat({k: pd.DataFrame(v) for k, v in rouge_results.items()})

precision    recall  fmeasure
rouge1    0   0.190231  0.273199  0.212075
          1   0.205862  0.289360  0.227292
          2   0.221325  0.306702  0.242738
rouge2    0   0.089399  0.109495  0.092696
          1   0.103130  0.125182  0.107020
          2   0.116431  0.140022  0.120220
rougeL    0   0.179613  0.256195  0.199689
          1   0.194480  0.272234  0.214000
          2   0.210147  0.289201  0.230084
rougeLsum 0   0.179952  0.254538  0.199219
          1   0.194424  0.271558  0.213692
          2   0.209767  0.287328  0.228237